# Importing Model

Huge credits to Chris10M on github!!

In [2]:
import sys
print(f"Kernel executable: {sys.executable}")

Kernel executable: /Users/sarath/Downloads/Projects/camhack/Vision/armVenv/bin/python


In [3]:
from triggerModel import EyeTrigger

torch_weights_path = '/Users/sarath/Downloads/Projects/camhack/Vision/Model Deployment/torch_weights.pth'
torch_blueprint = "my_model.onnx"
landmark_detection_path = "/Users/sarath/Downloads/Projects/camhack/Vision/open-eye-detector/models/68_face_landmarks_predictor.dat"
triggerModel = EyeTrigger(torch_blueprint, torch_weights_path, landmark_detection_path)

/Users/sarath/Downloads/Projects/camhack/Vision/armVenv/lib/python3.11/site-packages/face_recognition_models/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


## Side Quest: Port model from `keras` to `torch`

In [5]:
from keras.models import load_model
import torch
from onnx2pytorch import ConvertModel
import tf2onnx 
import onnx
from tensorflow import TensorSpec, float32

In [ ]:
weights_path = "/Users/sarath/Downloads/Projects/camhack/Vision/open-eye-detector/models/weights.149-0.01.hdf5"
model = load_model(weights_path, compile = False)
input_shape = (1,10,20,1)
spec = (TensorSpec(input_shape, float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(model,input_signature= spec, opset=13)

with open("my_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [8]:
onnx_model = onnx.load("my_model.onnx")
pytorch_model = ConvertModel(onnx_model)
pytorch_model.eval()

output_pytorch_path = '/Users/sarath/Downloads/Projects/camhack/Vision/Model Deployment/torch_weights.pth'
torch.save(pytorch_model.state_dict(), output_pytorch_path)


# Inference Testing

In [4]:
import cv2 as cv
from cameraController import CameraController

In [5]:
cam = cv.VideoCapture(1)

if not cam.isOpened():
    raise EnvironmentError("Camera was not able to be opened.")

controller = CameraController(cam)
try:
    while True:
        frameData = controller.getFrame()
        print(triggerModel.eyesOpen(*frameData))

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cam.release()
    cv.destroyAllWindows()

2025-11-02 12:28:26.471 Python[34207:15059032] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
True
False
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
False
True
True
True
True
False
True
True
True
True
False
False
False
False
False
False
True
False
False
False
True
False


KeyboardInterrupt: 